In [1]:
# Clone rag_faq folder from llm4gov repo
!rm -rf llm4gov
!git clone --filter=blob:none --sparse https://github.com/Labic-ICMC-USP/llm4gov.git
%cd llm4gov
!git sparse-checkout set rag_faq
%cd rag_faq
!ls

Cloning into 'llm4gov'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (282/282), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 282 (delta 78), reused 195 (delta 37), pack-reused 0 (from 0)
Receiving objects: 100% (282/282), 48.58 KiB | 12.15 MiB/s, done.
Resolving deltas: 100% (78/78), done.
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 1 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (1/1), 1.59 KiB | 1.59 MiB/s, done.
/content/llm4gov
remote: Enumerating objects: 189, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 189 (delta 62), reused 176 (delta 58), pack-reused 0 (from 0)
Receiving objects: 100% (189/189), 24.20 MiB | 13.38 MiB/s, done.
Resolving deltas: 100% (62/62), done.
Updating files: 100% (191/191), done.
/content/llm4gov/rag_faq
config.yaml		 projects	   Readme.md	      se

In [2]:
# Install requirements
!pip install -e .

Obtaining file:///content/llm4gov/rag_faq
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Running setup.py develop for rag-faq
ERROR: pip's dependency resolver does not currently take into account all the packages that are

## ⚙️ Configuração e carregamento de dados
Define os parâmetros essenciais para gerar FAQs e embeddings.

- **Configuração manual** (ajuste conforme necessário):
  - **csv_path**: Pasta com os arquivos CSV dos PPCs.
  - **course_name**: Nome do curso correspondente ao CSV.
  - **project_name**: Pasta onde serão salvos FAQs e embeddings.
  - **os.environ['OPENROUTER_API_KEY']**: Chave da API do OpenRouter.

In [6]:
import os, tempfile, yaml
import pandas as pd
from pathlib import Path
from rag_faq.config import load_config
from rag_faq.embedder import embed_faqs
from rag_faq.indexer import generate_faqs

# =============== EDIT THIS MANUALLY =============== #
csv_path = Path("data/ppp_bcc/ppp_bcc_chunks.csv")

course_name = "Bacharelado em Ciência da Computação"

project_name = "test_project"

os.environ['OPENROUTER_API_KEY'] = "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"
# ================================================== #

if not csv_path.exists():
    raise FileNotFoundError(f"CSV file not found: {csv_path}")

df = pd.read_csv(csv_path)
text_chunks = []
for _, row in df.iterrows():
    text_chunks.append({
        'text': row['text'],
        'chunk_id': row['chunk_id']
})

api_key = os.getenv("OPENROUTER_API_KEY")
if api_key is None:
    raise ValueError("OpenRouter API key not found in environment variables.")

# Cria arquivo de config temporário
config = load_config("config.yaml")
config["llm"]["faq_generator"]["api_key"] = api_key
config["llm"]["rag_answer"]["api_key"] = api_key
tmp_config = tempfile.NamedTemporaryFile(delete=False, suffix=".yaml")
tmp_config_path = tmp_config.name
tmp_config.close()
with open(tmp_config_path, "w", encoding="utf-8") as f:
    yaml.safe_dump(config, f, allow_unicode=True)

doc_stem = csv_path.parent.name
project_dir = Path(config["paths"]["projects_dir"]) / project_name / doc_stem
project_dir.mkdir(parents=True, exist_ok=True)

print(f"Project folder created at: {project_dir}")

Project folder created at: projects/test_project/ppp_bcc


## Geração de FAQs
Gera perguntas e respostas a partir dos PPCs/CSVs, usando prompts definidos para compor a base de conhecimento do RAG.

- **Saída**:
  - `individual/faq.csv` — FAQ de uma única persona
  - `unificado/faq_aluno.csv`, `unificado/faq_professor.csv`, `unificado/faq_pesquisador.csv` — FAQs por persona
  - `unificado/faq.csv` — FAQ combinado de todas as personas


### 🎯 Geração de FAQs (Persona única)
Gera FAQs para uma única persona (ex.: aluno).

- **Objetivo**:
  - Criar um conjunto de FAQs focado em um tipo específico de usuário
  - Salvar no subdiretório `individual`

- **Saída**: `individual/faq.csv` com perguntas e respostas voltadas à persona.


In [7]:
# Cria um diretório individual para FAQs de uma única persona
individual_dir = project_dir / "individual"
individual_dir.mkdir(parents=True, exist_ok=True)

persona = "aluno"
generate_faqs(config, individual_dir, text_chunks, course_name, persona)

Generating FAQs: 100%|██████████| 35/35 [07:16<00:00, 12.47s/it]

✅ FAQ saved to: projects/test_project/ppp_bcc/individual/faq.csv


In [8]:
# Exibe exemplos de dados do faq.csv
individual_df_data = pd.read_csv(f"{individual_dir}/faq.csv")

# Mostrar o cabeçalho do dataframe
display(individual_df_data.sample(5))
print(f"Total de perguntas: {len(individual_df_data)}")

# Exibe um exemplo de pergunta e resposta
if individual_df_data.empty:
    print("DataFrame vazio, sem exemplos para exibir.")
else:
    example_row = individual_df_data.iloc[10]

    print("\nExemplo de Q&A:")
    print(f"Q: {example_row['question']}")
    print(f"A: {example_row['answer']}")

,source_text,chunk_id,question,answer,course,persona
166,17 SCC0285 Análise de Séries Temporais e Aplic...,17,Qual é o objetivo da disciplina SCC0201 – Intr...,Apresentar ao aluno a conceituação teórica das...,Bacharelado em Ciência da Computação,aluno
269,"27 vagas do curso passou de 100 para 85, devid...",27,Em que anos houve uma queda da evasão nos últi...,"2021, 2022 e 2023",Bacharelado em Ciência da Computação,aluno
59,6 3. Metodologia do Curso A estrutura curricul...,6,Qual é o papel da Semana da Computação?,A Semana da Computação é um evento anual que p...,Bacharelado em Ciência da Computação,aluno
68,7 ● Aulas expositivas convencionais; ● Aulas e...,7,Quais são as oportunidades de internacionaliza...,Os alunos do ICMC têm à disposição uma comissã...,Bacharelado em Ciência da Computação,aluno
255,26 5. Evolução e Avaliação do Curso 5.1 Histór...,26,Qual é o número de vagas anuais oferecidas no ...,"A partir de 2021, o total de vagas do curso pa...",Bacharelado em Ciência da Computação,aluno


Total de perguntas: 350

Exemplo de Q&A:
Q: Qual é o objetivo principal do curso de Bacharelado em Ciências de Computação do ICMC USP?
A: Preparar um profissional com sólida formação conceitual, teórica e experimental em diferentes áreas de Computação.


### 👥 Geração de FAQs (Multi persona)
Gera FAQs para diferentes personas (aluno, professor, pesquisador).

- **Objetivo**:
  - Criar um conjunto de FAQs focado para cada persona com prompts dedicados
  - Salvar arquivos separados no subdiretório `unificado/`

- **Saídas** (em `unificado/`):
  - `faq_aluno.csv`
  - `faq_professor.csv`
  - `faq_pesquisador.csv`


In [12]:
# Cria um diretório unificado para FAQs de múltiplas personas
unificado_dir = project_dir / "unificado"
unificado_dir.mkdir(parents=True, exist_ok=True)

persona_type = ["aluno", "professor", "pesquisador"]

for persona in persona_type:
   generate_faqs(config, unificado_dir, text_chunks, course_name, persona, multi_persona=True)

Generating FAQs: 100%|██████████| 35/35 [07:07<00:00, 12.20s/it]


✅ FAQ saved to: projects/test_project/ppp_bcc/unificado/faq_aluno.csv


Generating FAQs: 100%|██████████| 35/35 [08:26<00:00, 14.47s/it]


✅ FAQ saved to: projects/test_project/ppp_bcc/unificado/faq_professor.csv


Generating FAQs: 100%|██████████| 35/35 [06:57<00:00, 11.92s/it]

✅ FAQ saved to: projects/test_project/ppp_bcc/unificado/faq_pesquisador.csv


In [13]:
# Agrupa FAQs específicas de todas personas em um único arquivo

# Lista de personas para agrupar
persona_files = ["faq_aluno.csv", "faq_professor.csv", "faq_pesquisador.csv"]
all_faqs = []

for file_name in persona_files:
    file_path = unificado_dir / file_name
    if file_path.exists():
        df = pd.read_csv(file_path)
        all_faqs.append(df)
        print(f"✅ Loaded {len(df)} FAQs from {file_name}")
    else:
        print(f"⚠️  File not found: {file_name}")

if all_faqs:
    # Concatena todos os dataframes
    merged_df = pd.concat(all_faqs, ignore_index=True)

    # Salva o arquivo
    merged_path = unificado_dir / "faq.csv"
    merged_df.to_csv(merged_path, index=False, encoding="utf-8")

    print(f"\n📊 Merge Summary:")
    print(f"📈 Total FAQs: {len(merged_df)}")

    # Mostra quantidade de FAQs por persona
    persona_counts = merged_df['persona'].value_counts()
    print(f"👥 FAQs by persona:")
    for persona, count in persona_counts.items():
        print(f"   - {persona}: {count}")

    # Mostra quantidade de FAQs por curso
    print(f"🎓 FAQs by course:")
    course_counts = merged_df['course'].value_counts()
    for course, count in course_counts.items():
        print(f"   - {course}: {count}")

    print(f"\n✅ Combined CSV saved to: {merged_path}")
else:
    print("❌ No FAQ files found to merge!")


✅ Loaded 350 FAQs from faq_aluno.csv
✅ Loaded 350 FAQs from faq_professor.csv
✅ Loaded 350 FAQs from faq_pesquisador.csv

📊 Merge Summary:
📈 Total FAQs: 1050
👥 FAQs by persona:
   - aluno: 350
   - professor: 350
   - pesquisador: 350
🎓 FAQs by course:
   - Bacharelado em Ciência da Computação: 1050

✅ Combined CSV saved to: projects/test_project/ppp_bcc/unificado/faq.csv


In [14]:
# Exibe exemplos de dados do faq.csv
unificado_df_data = pd.read_csv(f"{unificado_dir}/faq.csv")

# Mostrar o cabeçalho do dataframe
display(unificado_df_data.sample(5))
print(f"Total de perguntas: {len(unificado_df_data)}")

# Exibir um exemplo de pergunta e resposta
if unificado_df_data.empty:
    print("DataFrame vazio, sem exemplos para exibir.")
else:
    example_row = unificado_df_data.iloc[370]

    print("\nExemplo de Q&A:")
    print(f"Q: {example_row['question']}")
    print(f"A: {example_row['answer']}")

,source_text,chunk_id,question,answer,course,persona
368,2 1. Objetivos do Curso O curso de Bacharelado...,2,O curso prepara o estudante para atividades de...,"Sim, o egresso deverá estar apto a atuar em at...",Bacharelado em Ciência da Computação,professor
873,18 Melhorar o interesse dos alunos pelos curso...,18,Quais habilidades devem ser desenvolvidas nos ...,Conhecer a legislação vigente que regulamenta ...,Bacharelado em Ciência da Computação,pesquisador
879,18 Melhorar o interesse dos alunos pelos curso...,18,Quais são as habilidades que os alunos devem d...,Adquirir noções de análise de complexidade de ...,Bacharelado em Ciência da Computação,pesquisador
104,11 Familiarizar os estudantes com as várias es...,11,Quais são os conceitos abordados na disciplina...,"Conceitos de estatística, seus alcances e limi...",Bacharelado em Ciência da Computação,aluno
414,7 ● Aulas expositivas convencionais; ● Aulas e...,7,Quais são os benefícios da participação em eve...,A participação em eventos científicos incentiv...,Bacharelado em Ciência da Computação,professor


Total de perguntas: 1050

Exemplo de Q&A:
Q: Quais são as três áreas de atuação propostas para o egresso do curso de Bacharelado em Ciências de Computação?
A: O egresso pode atuar em: 1. Mercado de trabalho, englobando indústrias de computadores, empresas de software e setores de processamento de dados; 2. Atividades de pesquisa, em áreas específicas da computação ou que utilizam a computação como ferramenta; 3. Ações de empreendedorismo na área de computação.


## Geração de Embeddings
Cria representações vetoriais (embeddings) para todas as perguntas das FAQs, permitindo buscas por similaridade.

- **Saída**: `embeddings.npy` — arquivo com as representações vetoriais das perguntas.


In [18]:
# Gera embeddings somente se as respectivas etapas de geração de FAQ foram executadas
if unificado_dir:
    embed_faqs(config, unificado_dir)

if individual_dir:
    embed_faqs(config, individual_dir)

Generating embeddings: 100%|██████████| 1050/1050 [00:52<00:00, 19.82it/s]


✅ Embeddings saved to: projects/test_project/ppp_bcc/unificado


Generating embeddings: 100%|██████████| 350/350 [00:16<00:00, 21.23it/s]

✅ Embeddings saved to: projects/test_project/ppp_bcc/individual


## Criação de FAQs e embeddings via CLI
Da mesma maneira que foi feito manualmente a criação de FAQs e a geração dos embeddings, como mostrado nas células anteriores, é possível realizar esse processo diretamente via CLI (Command Line Interface).




In [11]:
# Via CLI:
# Persona única
# Mesmas configurações (BCC -  aluno - individual)
!python -m rag_faq.main --mode index --project test_project_cli --data-source ppp_bcc --persona aluno --index-mode individual --config {tmp_config_path}

2025-11-10 17:26:27.785071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762795587.843022    6124 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762795587.862692    6124 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762795587.912169    6124 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762795587.912235    6124 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762795587.912240    6124 computation_placer.cc:177] computation placer alr

In [16]:
# Via CLI:
# Multi-persona
# Mesmas configurações (BCC -  multi_persona - unificado)
!python -m rag_faq.main --mode index --project test_project_cli --data-source ppp_bcc --index-mode unificado --config {tmp_config_path}

2025-11-10 18:33:45.396922: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762799625.479365   22570 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762799625.499071   22570 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762799625.556853   22570 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762799625.556938   22570 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762799625.556944   22570 computation_placer.cc:177] computation placer alr

## Testes

### Teste 1 — FAQs e embeddings gerados neste notebook

Executa um teste rápido usando a base criada acima:
- Consulta na base individual (persona única: aluno)
- Consulta na base unificada (multi‑persona: aluno, professor, pesquisador)

In [20]:
# Consulta direta via funções (individual e unificado)

from rag_faq.generator import generate_rag_answer

question = "Como é organizada a grade curricular do Bacharelado em Ciência da Computação?"
print(f"Question: \n{question}\n")

print("Resposta individual (persona única: aluno):")
result_individual = generate_rag_answer(config, individual_dir, question)
print(result_individual["answer"])

print("\nResposta unificado (multi-persona: aluno, professor, pesquisador):")
result_unificado = generate_rag_answer(config, unificado_dir, question)
print(result_unificado["answer"])


Question: 
Como é organizada a grade curricular do Bacharelado em Ciência da Computação?

Resposta individual (persona única: aluno):
Resposta Final: A grade curricular do Bacharelado em Ciências de Computação é composta por disciplinas obrigatórias e optativas, divididas por semestres, com quantidade de créditos e indicação de pré-requisitos para cada disciplina. O curso é estruturado para que os alunos desenvolvam habilidades em diferentes áreas, como programação, modelagem de sistemas, desenvolvimento de software e hardware, e também para que adquiram conhecimento em áreas como inteligência artificial, bases de dados e engenharia de software.

Evidência de Apoio: A estrutura curricular é composta por disciplinas oferecidas por vários departamentos, incluindo Ciências de Computação, Sistemas de Computação, Matemática, Matemática Aplicada e Estatística, e Física. A grade curricular completa é mostrada no Apêndice 1 e o esquema da estrutura curricular é exibido no Apêndice 2.

Resposta

In [21]:
# Consulta via CLI (base individual)
!python -m rag_faq.main --mode query --project {project_name}/{doc_stem}/individual --config {tmp_config_path}

2025-11-10 19:11:38.961577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762801899.015376   31736 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762801899.043827   31736 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762801899.116864   31736 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762801899.116970   31736 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762801899.116980   31736 computation_placer.cc:177] computation placer alr

In [22]:
# Consulta via CLI (base unificada)
!python -m rag_faq.main --mode query --project {project_name}/{doc_stem}/unificado --config {tmp_config_path}


2025-11-10 19:12:43.771683: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762801963.840618   32004 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762801963.857794   32004 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762801963.899524   32004 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762801963.899601   32004 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762801963.899607   32004 computation_placer.cc:177] computation placer alr

### Teste 2 — FAQs e embeddings pré‑gerados

Executa um teste usando bases já existentes, combinando embeddings de todos os cursos do ICMC.
  - Consulta na base individual (persona única)
  - Consulta na base unificada (multi‑persona)

In [24]:
# Consulta direta via funções (individual e unificado)

from rag_faq.generator import generate_rag_answer

project_name = "batch_proj"
doc_stem = "all_courses"
project_dir = os.path.join(config["paths"]["projects_dir"], f"{project_name}/{doc_stem}")

all_courses_individual_dir = project_dir + "/individual"
all_courses_unificado_dir = project_dir + "/unificado"

question = "Quais competências específicas se espera que os egressos da Licenciatura em Matemática possuam?"
print(f"Question: \n{question}\n")

print("Resposta individual (persona única: aluno):")
result_individual = generate_rag_answer(config, all_courses_individual_dir, question)
print(result_individual["answer"])

print("\nResposta unificado (multi-persona: aluno, professor, pesquisador):")
result_unificado = generate_rag_answer(config, all_courses_unificado_dir, question)
print(result_unificado["answer"])

Question: 
Quais competências específicas se espera que os egressos da Licenciatura em Matemática possuam?

Resposta individual (persona única: aluno):
Resposta Final: 
As competências específicas necessárias à formação do professor de Matemática que se espera que os egressos da Licenciatura em Matemática possuam incluem:

1. Atuar com base numa visão abrangente do papel social do educador e do papel da Matemática como campo do conhecimento humano;
2. Exercer a reflexão crítica sobre sua própria prática como educador, sendo capaz de buscar e compreender novas ideias e novas tecnologias, relacionando-as ao ensino de Matemática;
3. Trabalhar em equipe, visualizando dimensões multidisciplinares dos conteúdos relacionados à Matemática;
4. Analisar criticamente materiais didáticos de Matemática (livros, softwares com finalidades didáticas, etc.) e elaborar propostas alternativas para a sala de aula;
5. Compreender aspectos históricos e sociológicos da Matemática e como estes se relacionam a

In [25]:
# Consulta via CLI (base individual)
!python -m rag_faq.main --mode query --project {project_name}/{doc_stem}/individual --config {tmp_config_path}

2025-11-10 19:17:31.402075: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762802251.454247   33176 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762802251.466922   33176 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762802251.496890   33176 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762802251.496943   33176 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762802251.496947   33176 computation_placer.cc:177] computation placer alr

In [26]:
# Consulta via CLI (base unificada)
!python -m rag_faq.main --mode query --project {project_name}/{doc_stem}/unificado --config {tmp_config_path}

2025-11-10 19:18:13.161260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762802293.185741   33362 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762802293.192957   33362 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762802293.210816   33362 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762802293.210866   33362 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762802293.210871   33362 computation_placer.cc:177] computation placer alr